# nnU-Net V2

# 1. Import Packages for Google Colab

In [1]:
# Import basic packages for later use
import os
import shutil
from collections import OrderedDict

import json
import matplotlib.pyplot as plt
import nibabel as nib

import numpy as np
import torch

In [2]:
# check whether GPU accelerated computing is available
assert torch.cuda.is_available() # if there is an error here, enable GPU in the Runtime

# 2. Installing nnU-Net

In [2]:
# install nnunet - yes it is that easy
!pip install nnunetv2


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.6/178.6 kB 3.6 MB/s eta 0:00:00a 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.7/61.7 kB 7.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.7/52.7 MB 23.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.3/37.3 MB 41.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 97.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.4/96.4 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 83.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 76.5 MB/s eta 0:00:00
  Created

**Note**: you do not have to restart the Runtime even when the following error appears, this is simply due to a reinstallation of a package - so no worries.


```
WARNING: The following packages were previously imported in this runtime:

[argparse]

You must restart the runtime in order to use newly installed versions.
```

# 3. Verifying installation of nn-Unet

In [3]:
# check if nnunet can be imported
import nnunetv2


You should read the following if the installation was successful:


```
Please cite the following paper when using nnUNet:

Isensee, F., Jaeger, P.F., Kohl, S.A.A. et al. "nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation." Nat Methods (2020). https://doi.org/10.1038/s41592-020-01008-z


If you have questions or suggestions, feel free to open an issue at https://github.com/MIC-DKFZ/nnUNet

```



## 3.1 Installation and initialization of Weights&Biases (non-mandatory)
This is a package which allows you to easily visualize metrics during your training.

We will use this here to get access to System Information (GPU utilization etc.)when running on Google Colab - however it is also useful to use at a local workstation.

To use Weights&Biases you will need an account.
This can be created at https://wandb.ai

During the initialization with ```wandb.init()``` it will ask you for an API key, which you can obtain from:
https://wandb.ai/settings
under API keys

In [7]:
# !pip install wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.1/254.1 kB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 9.0 MB/s eta 0:00:00


In [ ]:
# import wandb
# wandb.init(project="nnU-Net_Workshop")

<IPython.core.display.Javascript object>

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


### 3.2 Installation of GDOWN (only for in-person workshops)
GDown allows to download files from GoogleDrive with Python.
Therefore it is used for in-person workshops.

In [ ]:
# install gdown to download files from GoogleDrive
# !pip install gdown

# 4. Connect Google Colab with GoogleDrive
This is heavily encouraged when using Google Colab, otherwise loading, accessing and saving of data checkpoints etc. is just much harder (if not impossible).

**Note**:
When ```drive.mount()``` is executed, there will appear a popup window, which will ask you which Google Account should be connected and then for permission regarding Colab accessing your GoogleDrive.

There can be some problems at this step, for me the solution was to Download the GoogleDrive Application.
You can download it from: https://www.google.com/drive/download/


If there are any problems at this stage, feel free to contact me (during the workshop).

In [4]:
# for colab users only - mounting the drive

from google.colab import drive
drive.mount('/content/drive',force_remount = True)

drive_dir = "/content/drive/My Drive"
mount_dir = os.path.join(drive_dir, "Colab Notebooks")
base_dir = os.getcwd()

Mounted at /content/drive


In [5]:
assert os.path.exists(drive_dir) # if this fails, something went wrong with mounting GoogleDrive
if os.path.exists(mount_dir) is False:
    os.makedirs(mount_dir)

# 5. Setting up nnU-Nets folder structure and environment variables
nnUnet expects a certain folder structure and environment variables.

Roughly they tell nnUnet:
1. Where to look for stuff
2. Where to put stuff

For more information about this please check: https://github.com/MIC-DKFZ/nnUNet/blob/master/documentation/setting_up_paths.md

In [7]:
def make_if_dont_exist(folder_path,overwrite=False):
    """
    creates a folder if it does not exists
    input:
    folder_path : relative path of the folder which needs to be created
    over_write :(default: False) if True overwrite the existing folder
    """
    if os.path.exists(folder_path):

        if not overwrite:
            print(f"{folder_path} exists.")
        else:
            print(f"{folder_path} overwritten")
            shutil.rmtree(folder_path)
            os.makedirs(folder_path)

    else:
      os.makedirs(folder_path)
      print(f"{folder_path} created!")

## 5.1 Set environment Variables and creating folders

In [8]:
# Maybe move path of preprocessed data directly on content - this may be signifcantely faster!
print("Current Working Directory {}".format(os.getcwd()))
path_dict = {
    "nnUNet_raw" : os.path.join(mount_dir, "nnUNet_raw"),
    "nnUNet_preprocessed" : os.path.join(mount_dir, "nnUNet_preprocessed"), # 1 experiment: 1 epoch took 112s
    "nnUNet_results" : os.path.join(mount_dir, "nnUNet_results"),
    "RAW_DATA_PATH" : os.path.join(mount_dir, "RawData"), # This is used here only for convenience (not necessary for nnU-Net)!
}

# Write paths to environment variables
for env_var, path in path_dict.items():
  os.environ[env_var] = path

# Check whether all environment variables are set correct!
for env_var, path in path_dict.items():
  if os.getenv(env_var) != path:
    print("Error:")
    print("Environment Variable {} is not set correctly!".format(env_var))
    print("Should be {}".format(path))
    print("Variable is {}".format(os.getenv(env_var)))
  make_if_dont_exist(path, overwrite=False)

print("If No Error Occured Continue Forward. =)")

Current Working Directory /content
/content/drive/My Drive/Colab Notebooks/nnUNet_raw exists.
/content/drive/My Drive/Colab Notebooks/nnUNet_preprocessed exists.
/content/drive/My Drive/Colab Notebooks/nnUNet_results exists.
/content/drive/My Drive/Colab Notebooks/RawData exists.
If No Error Occured Continue Forward. =)


# 6. Using nnU-Net on our own Pituitary tumor dataset

## 6.1 Preprocessing
The dataset doesn't satisfy the requirement of nnUNet if this part has any error

The dataset is named as " Dataset006_Pituitarytumor "

In [19]:
!nnUNetv2_plan_and_preprocess -d 6 --verify_dataset_integrity

Fingerprint extraction...
Dataset006_Pituitarytumor
Using <class 'nnunetv2.imageio.simpleitk_reader_writer.SimpleITKIO'> as reader/writer
Origin images: (-113.6096420288086, -56.61311721801758, -125.25467681884766). 
Origin seg: (-113.6102294921875, -56.61252212524414, -125.25479125976562). 
Image files: ['/content/drive/My Drive/Colab Notebooks/nnUNet_raw/Dataset006_Pituitarytumor/imagesTr/Pituitarytumor_128_0000.nii.gz']. 
Seg file: /content/drive/My Drive/Colab Notebooks/nnUNet_raw/Dataset006_Pituitarytumor/labelsTr/Pituitarytumor_128.nii.gz

Direction images: (0.9985552422625675, 0.053529908811431486, -0.0047059757109088945, -0.01630992307008615, 0.38536931872355656, 0.9226182591794582, -0.05119975155596907, 0.9212084655761592, -0.385685625375479). 
Direction seg: (0.9985551581940791, 0.05352993275607706, -0.004705995068746602, -0.0163105026527398, 0.38536923450915894, 0.9226182968816656, -0.051201206501532356, 0.921208499414196, -0.38568553494989616). 
Image files: ['/content/driv

### 6.2 Training nnU-Net
here we will train a 3D nnU-Net on Full Resolution

In [ ]:
# train the 3d nnUnet on the Full Resolution with Dataset 4 and Cross Validation Split 0
# run one epoch for a test
# !nnUNetv2_train 4 3d_fullres 0 -tr nnUNetTrainer_1epoch




Please cite the following paper when using nnUNet:

Isensee, F., Jaeger, P.F., Kohl, S.A.A. et al. "nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation." Nat Methods (2020). https://doi.org/10.1038/s41592-020-01008-z


If you have questions or suggestions, feel free to open an issue at https://github.com/MIC-DKFZ/nnUNet

###############################################
I am running the following nnUNet: 3d_fullres
My trainer class is:  <class 'nnunet.training.network_training.nnUNetTrainerV2.nnUNetTrainerV2'>
For that I will be using the following configuration:
num_classes:  2
modalities:  {0: 'MRI'}
use_mask_for_norm OrderedDict([(0, False)])
keep_only_largest_region None
min_region_size_per_class None
min_size_per_class None
normalization_schemes OrderedDict([(0, 'nonCT')])
stages...

stage:  0
{'batch_size': 9, 'num_pool_per_axis': [3, 3, 3], 'patch_size': array([40, 56, 40]), 'median_patient_size_in_voxels': array([36, 50, 35]), 'current_spaci

In [25]:
# run 1 epoch for a test
# !nnUNetv2_train 6 3d_fullres 3 -tr nnUNetTrainer_1epoch

Using device: cuda:0

#######################################################################
Please cite the following paper when using nnU-Net:
Isensee, F., Jaeger, P. F., Kohl, S. A., Petersen, J., & Maier-Hein, K. H. (2021). nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation. Nature methods, 18(2), 203-211.
#######################################################################


This is the configuration used by this training:
Configuration name: 3d_fullres
 {'data_identifier': 'nnUNetPlans_3d_fullres', 'preprocessor_name': 'DefaultPreprocessor', 'batch_size': 2, 'patch_size': [12, 448, 320], 'median_image_size_in_voxels': [15.0, 499.0, 426.0], 'spacing': [3.299999952316284, 0.44921875, 0.44921875], 'normalization_schemes': ['ZScoreNormalization'], 'use_mask_for_norm': [False], 'UNet_class_name': 'PlainConvUNet', 'UNet_base_num_features': 32, 'n_conv_per_stage_encoder': [2, 2, 2, 2, 2, 2, 2], 'n_conv_per_stage_decoder': [2, 2, 2, 2, 2, 2], 'nu

In [20]:
!nnUNetv2_train 6 3d_fullres 4 --npz  # fold 4

Using device: cuda:0

#######################################################################
Please cite the following paper when using nnU-Net:
Isensee, F., Jaeger, P. F., Kohl, S. A., Petersen, J., & Maier-Hein, K. H. (2021). nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation. Nature methods, 18(2), 203-211.
#######################################################################


This is the configuration used by this training:
Configuration name: 3d_fullres
 {'data_identifier': 'nnUNetPlans_3d_fullres', 'preprocessor_name': 'DefaultPreprocessor', 'batch_size': 2, 'patch_size': [14, 384, 320], 'median_image_size_in_voxels': [17.0, 493.5, 427.0], 'spacing': [3.299999952316284, 0.44921875, 0.44921875], 'normalization_schemes': ['ZScoreNormalization'], 'use_mask_for_norm': [False], 'UNet_class_name': 'PlainConvUNet', 'UNet_base_num_features': 32, 'n_conv_per_stage_encoder': [2, 2, 2, 2, 2, 2, 2], 'n_conv_per_stage_decoder': [2, 2, 2, 2, 2, 2], 'nu

In [9]:
!nnUNetv2_train 6 3d_fullres 2 --npz  # fold 2

Using device: cuda:0

#######################################################################
Please cite the following paper when using nnU-Net:
Isensee, F., Jaeger, P. F., Kohl, S. A., Petersen, J., & Maier-Hein, K. H. (2021). nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation. Nature methods, 18(2), 203-211.
#######################################################################


This is the configuration used by this training:
Configuration name: 3d_fullres
 {'data_identifier': 'nnUNetPlans_3d_fullres', 'preprocessor_name': 'DefaultPreprocessor', 'batch_size': 2, 'patch_size': [14, 384, 320], 'median_image_size_in_voxels': [17.0, 493.5, 427.0], 'spacing': [3.299999952316284, 0.44921875, 0.44921875], 'normalization_schemes': ['ZScoreNormalization'], 'use_mask_for_norm': [False], 'UNet_class_name': 'PlainConvUNet', 'UNet_base_num_features': 32, 'n_conv_per_stage_encoder': [2, 2, 2, 2, 2, 2, 2], 'n_conv_per_stage_decoder': [2, 2, 2, 2, 2, 2], 'nu

In [31]:
# !nnUNetv2_train 6 3d_fullres 3 --val --npz

Using device: cuda:0

#######################################################################
Please cite the following paper when using nnU-Net:
Isensee, F., Jaeger, P. F., Kohl, S. A., Petersen, J., & Maier-Hein, K. H. (2021). nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation. Nature methods, 18(2), 203-211.
#######################################################################

2023-12-26 07:48:23.179482: Using splits from existing split file: /content/drive/My Drive/Colab Notebooks/nnUNet_preprocessed/Dataset006_Pituitarytumor/splits_final.json
2023-12-26 07:48:23.197123: The split file contains 5 splits.
2023-12-26 07:48:23.201375: Desired fold for training: 3
2023-12-26 07:48:23.204096: This split has 36 training and 9 validation cases.
2023-12-26 07:48:23.209451: predicting Pituitarytumor_21
2023-12-26 07:48:42.014847: predicting Pituitarytumor_22
2023-12-26 07:48:48.891091: predicting Pituitarytumor_23
2023-12-26 07:48:59.248104: predicti

In [ ]:
# !nnUNetv2_train 6 3d_fullres 4 --val --npz

In [40]:
# !nnUNetv2_find_best_configuration 6 -c 2d

Traceback (most recent call last):
  File "/usr/local/bin/nnUNetv2_find_best_configuration", line 8, in <module>
    sys.exit(find_best_configuration_entry_point())
  File "/usr/local/lib/python3.10/dist-packages/nnunetv2/evaluation/find_best_configuration.py", line 295, in find_best_configuration_entry_point
    find_best_configuration(dataset_name, model_dict, allow_ensembling=not args.disable_ensembling,
  File "/usr/local/lib/python3.10/dist-packages/nnunetv2/evaluation/find_best_configuration.py", line 100, in find_best_configuration
    accumulate_cv_results(output_folder, merged_output_folder, folds, num_processes, overwrite)
  File "/usr/local/lib/python3.10/dist-packages/nnunetv2/evaluation/accumulate_cv_results.py", line 36, in accumulate_cv_results
    raise RuntimeError(f"fold {f} of model {trained_model_folder} is missing. Please train it!")
RuntimeError: fold 0 of model /content/drive/My Drive/Colab Notebooks/nnUNet_results/Dataset006_Pituitarytumor/nnUNetTrainer__nnUNetP

#  7 Inference

nnUNet_find_best_configuration will print inference commands you need to use. The easiest way to run inference is to simply use these commands.

For each of the desired configurations, run:

```nnUNet_predict -i INPUT_FOLDER -o OUTPUT_FOLDER -d DATASET_NAME_OR_ID -m CONFIGURATION --save_npz```

Only specify ```--save_npz``` if you intend to use ensembling. ```--save_npz``` will make the command save the softmax probabilities alongside of the predicted segmentation masks requiring a lot of disk space.

Note: Please select a separate OUTPUT_FOLDER for each configuration!

In [9]:
# use fully trained nnU-Net to make predictions on data
# !nnUNetv2_predict -i "${nnUNet_raw}/Dataset006_Pituitarytumor/imagesTs/" -o "${nnUNet_results}/Dataset006_Pituitarytumor/predTs/" -d 6 -c 2d -f 2


#######################################################################
Please cite the following paper when using nnU-Net:
Isensee, F., Jaeger, P. F., Kohl, S. A., Petersen, J., & Maier-Hein, K. H. (2021). nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation. Nature methods, 18(2), 203-211.
#######################################################################

Traceback (most recent call last):
  File "/usr/local/bin/nnUNetv2_predict", line 8, in <module>
    sys.exit(predict_entry_point())
  File "/usr/local/lib/python3.10/dist-packages/nnunetv2/inference/predict_from_raw_data.py", line 833, in predict_entry_point
    predictor.initialize_from_trained_model_folder(
  File "/usr/local/lib/python3.10/dist-packages/nnunetv2/inference/predict_from_raw_data.py", line 86, in initialize_from_trained_model_folder
    checkpoint = torch.load(join(model_training_output_dir, f'fold_{f}', checkpoint_name),
  File "/usr/local/lib/python3.10/dist-packages/tor

In [12]:
# use fully trained nnU-Net to make predictions on data
!nnUNetv2_predict -i "${nnUNet_raw}/Dataset006_Pituitarytumor/imagesTs/" -o "${nnUNet_results}/Dataset006_Pituitarytumor/predTs/" -d 6 -c 3d_fullres -f 2


#######################################################################
Please cite the following paper when using nnU-Net:
Isensee, F., Jaeger, P. F., Kohl, S. A., Petersen, J., & Maier-Hein, K. H. (2021). nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation. Nature methods, 18(2), 203-211.
#######################################################################

There are 12 cases in the source folder
I am process 0 out of 1 (max process ID is 0, we start counting with 0!)
There are 12 cases that I would like to predict

Predicting Pituitarytumor_0:
perform_everything_on_gpu: True
100% 12/12 [00:04<00:00,  2.84it/s]
Prediction done, transferring to CPU if needed
sending off prediction to background worker for resampling and export
done with Pituitarytumor_0

Predicting Pituitarytumor_1:
perform_everything_on_gpu: True
100% 20/20 [00:06<00:00,  3.04it/s]
Prediction done, transferring to CPU if needed
sending off prediction to background worker for r

In [16]:
# use fully trained nnU-Net to make predictions on data
!nnUNetv2_predict -i "${nnUNet_raw}/Dataset005_Prostate/imagesTs/" -o "${nnUNet_results}/Dataset005_Prostate/predTs/" -d 6 -c 3d_fullres


#######################################################################
Please cite the following paper when using nnU-Net:
Isensee, F., Jaeger, P. F., Kohl, S. A., Petersen, J., & Maier-Hein, K. H. (2021). nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation. Nature methods, 18(2), 203-211.
#######################################################################

Traceback (most recent call last):
  File "/usr/local/bin/nnUNetv2_predict", line 8, in <module>
    sys.exit(predict_entry_point())
  File "/usr/local/lib/python3.10/dist-packages/nnunetv2/inference/predict_from_raw_data.py", line 833, in predict_entry_point
    predictor.initialize_from_trained_model_folder(
  File "/usr/local/lib/python3.10/dist-packages/nnunetv2/inference/predict_from_raw_data.py", line 86, in initialize_from_trained_model_folder
    checkpoint = torch.load(join(model_training_output_dir, f'fold_{f}', checkpoint_name),
  File "/usr/local/lib/python3.10/dist-packages/tor

In [ ]:
# Optional
!nnUNetv2_find_best_configuration -d 6

**Note**:

If you interrupted the training go the given fold inside of the RESULTS_FOLDER for the task and:
1. rename **model_best.model.pkl** to **model_final_checkpoint.model.pkl**
2. rename **model_best.model** to **model_final_checkpoint.model**

If you wish to run ensembling, you can ensemble the predictions from several configurations with the following command:

```nnUNet_ensemble -f FOLDER1 FOLDER2 ... -o OUTPUT_FOLDER -pp POSTPROCESSING_FILE```